## Processing NYC taxi data using Ray Datasets¶

© 2019-2022, Anyscale. All Rights Reserved

📖 [Back to Table of Contents](../ex_00_tutorial_overview.ipynb)<br>

The [NYC Taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page) is a popular tabular dataset. In this example, we demonstrate some basic data processing on this dataset using Ray Datasets.

### Learning Objectives

This tutorial will cover:

* reading Parquet data from an external source
* inspecting the metadata and first few rows of a large Ray Dataset
* calculating some common global and grouped statistics on the datase
* dropping columns and rows and ddding a derived column
* shuffling the dataset
* sharding the dataset and feeding it to parallel consumers (trainers)
* applying batch (offline) inference to the data

Let's get started by importing some modules and starting our Ray
cluster

In [14]:
import logging, random
import ray
import os
import warnings

In [15]:
warnings.filterwarnings("ignore")
os.environ["PYTHONWARNINGS"] = "ignore"

In [16]:
if ray.is_initialized:
    ray.shutdown()
ray.init(logging_level=logging.ERROR)

Python version:,3.8.13
Ray version:,2.0.0rc0
Dashboard:,http://127.0.0.1:8266


### Reading and Inspecting the Data

Next, we read a few of the files from the dataset. This read is semi-lazy, where reading of the first file is eagerly executed, but reading of all other files is delayed until the underlying data is needed by downstream operations (e.g., consuming the data with `ds.take()`, or transforming the data with `ds.map_batches())`.



In [17]:
# Read two Parquet files in parallel.
ds = ray.data.read_parquet([
    "s3://ursa-labs-taxi-data/2009/01/data.parquet",
    "s3://ursa-labs-taxi-data/2009/02/data.parquet",
])

We can easily inspect the schema of this dataset. For Parquet files, we don’t even have to read the actual data to get the schema; we can read it from the lightweight Parquet metadata!

In [18]:
# Fetch the schema from the underlying Parquet metadata.
ds.schema()

vendor_id: string
pickup_at: timestamp[us]
dropoff_at: timestamp[us]
passenger_count: int8
trip_distance: float
pickup_longitude: float
pickup_latitude: float
rate_code_id: null
store_and_fwd_flag: string
dropoff_longitude: float
dropoff_latitude: float
payment_type: string
fare_amount: float
extra: float
mta_tax: float
tip_amount: float
tolls_amount: float
total_amount: float
-- schema metadata --
pandas: '{"index_columns": [{"kind": "range", "name": null, "start": 0, "' + 2527

Parquet even stores the number of rows per file in the Parquet metadata, so we can get the number of rows in `ds` without triggering a full data read.

In [19]:
ds.count()

27472535

We can get a nice, cheap summary of the `Dataset` by leveraging its informative repr:

In [20]:
# Display some metadata about the dataset.
ds

Dataset(num_blocks=2, num_rows=27472535, schema={vendor_id: string, pickup_at: timestamp[us], dropoff_at: timestamp[us], passenger_count: int8, trip_distance: float, pickup_longitude: float, pickup_latitude: float, rate_code_id: null, store_and_fwd_flag: string, dropoff_longitude: float, dropoff_latitude: float, payment_type: string, fare_amount: float, extra: float, mta_tax: float, tip_amount: float, tolls_amount: float, total_amount: float})

We can also poke at the actual data, taking a peek at a single row. Since this is only returning a row from the first file, reading of the second file is not triggered yet.


In [21]:
ds.take(1)

[ArrowRow({'vendor_id': 'VTS',
           'pickup_at': datetime.datetime(2009, 1, 4, 2, 52),
           'dropoff_at': datetime.datetime(2009, 1, 4, 3, 2),
           'passenger_count': 1,
           'trip_distance': 2.630000114440918,
           'pickup_longitude': -73.99195861816406,
           'pickup_latitude': 40.72156524658203,
           'rate_code_id': None,
           'store_and_fwd_flag': None,
           'dropoff_longitude': -73.99380493164062,
           'dropoff_latitude': 40.6959228515625,
           'payment_type': 'CASH',
           'fare_amount': 8.899999618530273,
           'extra': 0.5,
           'mta_tax': None,
           'tip_amount': 0.0,
           'tolls_amount': 0.0,
           'total_amount': 9.399999618530273})]

To get a better sense of the data size, we can calculate the size in bytes of the full dataset. Note that for Parquet files, this size-in-bytes will be pulled from the Parquet metadata (not triggering a data read) and will therefore be the on-disk size of the data; this might be significantly smaller than the in-memory size!

**Note**: Datasets will only read one file eagerly, which allows us to inspect a subset of the data without having to read the entire dataset.

In [22]:
ds.size_bytes()

4485652320

In order to get the in-memory size, we can trigger full reading of the dataset and inspect the size in bytes.

In [23]:
ds.fully_executed().size_bytes()

Read progress: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [02:15<00:00, 67.63s/it]


2263031675

### Data Exploration and Cleaning¶

Let’s calculate some stats to get a better picture of our data.


In [24]:
# What's the longest trip distance, largest tip amount, and most number of passengers?
ds.max(["trip_distance", "tip_amount", "passenger_count"])

Shuffle Reduce: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 259.68it/s]


ArrowRow({'max(trip_distance)': 50.0,
          'max(tip_amount)': 100.0,
          'max(passenger_count)': 113})

Whoa, looking at the results above, there was a trip with **113** people in the taxi!? Let’s check out these kind of many-passenger records by filtering to just these records using our `ds.map_batches()` batch mapping API.

**Note**: Our filtering UDF receives a Pandas DataFrame, which is the default batch format for tabular data, and returns a Pandas DataFrame, which keeps the Dataset in a tabular format.

In [25]:
# Whoa, 113 passengers? I need to see this record and other ones with lots of passengers.
ds.map_batches(lambda df: df[df["passenger_count"] > 10]).take()

Map_Batches: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:05<00:00,  2.91s/it]


[PandasRow({'vendor_id': 'VTS',
            'pickup_at': Timestamp('2009-01-22 11:47:00'),
            'dropoff_at': Timestamp('2009-01-22 12:00:00'),
            'passenger_count': 113,
            'trip_distance': 0.0,
            'pickup_longitude': 3555.912841796875,
            'pickup_latitude': 935.5253295898438,
            'rate_code_id': None,
            'store_and_fwd_flag': None,
            'dropoff_longitude': -74.01129913330078,
            'dropoff_latitude': 1809.957763671875,
            'payment_type': 'CASH',
            'fare_amount': 13.300000190734863,
            'extra': 0.0,
            'mta_tax': nan,
            'tip_amount': 0.0,
            'tolls_amount': 0.0,
            'total_amount': 13.300000190734863})]

That seems weird, probably bad data, or at least data points that I’m not interested in. We should filter these out!

In [26]:
# Filter out all records with over 10 passengers.
ds = ds.map_batches(lambda df: df[df["passenger_count"] <= 10])

Map_Batches: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:33<00:00, 16.77s/it]


We don’t have any use for the store_and_fwd_flag or mta_tax columns, so let’s drop those.

In [27]:
# Drop some columns.
ds = ds.map_batches(lambda df: df.drop(columns=["store_and_fwd_flag", "mta_tax"]))

Map_Batches: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:37<00:00, 18.75s/it]


Let’s say we want to know how many trips there are for each passenger count. This can be done using `.groupby()` 

In [28]:
ds.groupby("passenger_count").count().take()

Shuffle Reduce: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 12.15it/s]


[PandasRow({'passenger_count': -127, 'count()': 2}),
 PandasRow({'passenger_count': -48, 'count()': 45}),
 PandasRow({'passenger_count': 0, 'count()': 794}),
 PandasRow({'passenger_count': 1, 'count()': 18634337}),
 PandasRow({'passenger_count': 2, 'count()': 4503747}),
 PandasRow({'passenger_count': 3, 'count()': 1196381}),
 PandasRow({'passenger_count': 4, 'count()': 559279}),
 PandasRow({'passenger_count': 5, 'count()': 2452176}),
 PandasRow({'passenger_count': 6, 'count()': 125773})]

Again, it looks like there are some more nonsensical passenger counts, i.e., the negative ones. Let’s filter those out too.

In [22]:
# Filter out all records with over 10 passengers.
ds = ds.map_batches(lambda df: df[df["passenger_count"] <= 10])

Map_Batches: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:46<00:00, 23.49s/it]


### Projection (selection) and filter pushdown

Note that Ray Datasets’ Parquet reader supports projection (column selection) and row filter pushdown, where we can push the above column selection and the row-based filter to the Parquet read. If we specify column selection at Parquet read time, the unselected columns won’t even be read from disk!

The row-based filter is specified via [Arrow’s dataset field expressions](https://arrow.apache.org/docs/6.0/python/generated/pyarrow.dataset.Expression.html#pyarrow.dataset.Expression). See the [feature guide for reading Parquet data](https://docs.ray.io/en/master/data/creating-datasets.html#dataset-supported-file-formats) for more information.

In [29]:
# Only read the passenger_count and trip_distance columns.
import pyarrow as pa

filter_expr = (
    (pa.dataset.field("passenger_count") <= 10)
    & (pa.dataset.field("passenger_count") > 0)
)

pushdown_ds = ray.data.read_parquet(
    [
        "s3://ursa-labs-taxi-data/2009/01/data.parquet",
        "s3://ursa-labs-taxi-data/2009/02/data.parquet",
    ],
    columns=["passenger_count", "trip_distance"],
    filter=filter_expr,
)

# Force full execution of both of the file reads.
pushdown_ds = pushdown_ds.fully_executed()
pushdown_ds

Read progress: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:09<00:00,  4.78s/it]


Dataset(num_blocks=2, num_rows=27471693, schema={passenger_count: int8, trip_distance: float})

In [30]:
# Delete the pushdown dataset. Deleting the Dataset object
# will release the underlying memory in the cluster. This was only for ilustration of pushdown functionality.
del pushdown_ds

Do the passenger counts influences the typical trip distance?

In [ ]:
# Mean trip distance grouped by passenger count.
ds.groupby("passenger_count").mean("trip_distance").take()

### Ingesting into Model Trainers

Now that we’ve learned more about our data and we have cleaned up our dataset a bit, we now look at how we can feed this dataset into some dummy model trainers.

First, let’s do a full global random shuffle of the dataset to decorrelate these samples.

In [ ]:
ds = ds.random_shuffle()

#### Create a model trainer

We define a dummy `Trainer` actor, where each trainer will consume a dataset shard in batches and simulate model training.

**Note**: In a real training workflow, we would feed `ds` to Ray Train, which would do this sharding and creation of training actors for us, under the hood.

In [ ]:
@ray.remote
class Trainer:
    def __init__(self, rank: int):
        pass

    def train(self, shard: ray.data.Dataset) -> int:
        for batch in shard.iter_batches(batch_size=256):
            pass
        return shard.count()

trainers = [Trainer.remote(i) for i in range(4)]
trainers

Next, we split the dataset into `len(trainers)` shards, ensuring that the shards are of equal size, and providing the trainer actor handles to Ray Datasets as locality hints, so Datasets can try to colocate shard data with trainers in order to decrease data movement.

In [ ]:
shards = ds.split(n=len(trainers), equal=True, locality_hints=trainers)
shards

Finally, we simulate training, passing each shard to the corresponding trainer. The number of rows per shard is returned.

In [ ]:
ray.get([w.train.remote(s) for w, s in zip(trainers, shards)])

In [ ]:
# Delete trainer actor handle references, which should terminate the actors.
del trainers

#### Parallel Batch Inference

After we’ve trained a model, we may want to perform batch (offline) inference on such a tabular dataset. With Ray Datasets, this is as easy as a `ds.map_batches()` call!

First, we define a callable class that will cache the loading of the model in its constructor.

In [ ]:
import pandas as pd

def load_model():
    # A dummy model.This could be loaded from a model registry or checkpoint
    def model(batch: pd.DataFrame) -> pd.DataFrame:
        return pd.DataFrame({"score": batch["passenger_count"] % 2 == 0})
    
    return model

class BatchInferModel:
    def __init__(self):
        self.model = load_model()
    def __call__(self, batch: pd.DataFrame) -> pd.DataFrame:
        return self.model(batch)

`BatchInferModel`’s constructor will only be called once per actor worker when using the actor pool compute strategy in `ds.map_batches()`.



In [ ]:
ds.map_batches(BatchInferModel, batch_size=2048, compute="actors").take(5)

#### Auto scaling batch inferences 

We can also configure the autoscaling actor pool that this inference stage uses, setting upper and lower bounds on the actor pool size, and even tweak the batch prefetching vs. inference task queueing tradeoff.

In [38]:
from ray.data import ActorPoolStrategy

# The actor pool will have at least 2 workers and at most 8 workers.
strategy = ActorPoolStrategy(min_size=2, max_size=8)

ds.map_batches(
    BatchInferModel,
    batch_size=256,
    #num_gpus=1,  # Uncomment this to run this on GPUs!
    compute=strategy,
).take(5)

Map Progress (8 actors 0 pending): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [02:03<00:00, 61.62s/it]


[PandasRow({'score': False}),
 PandasRow({'score': True}),
 PandasRow({'score': True}),
 PandasRow({'score': False}),
 PandasRow({'score': False})]

📖 [Back to Table of Contents](../ex_00_tutorial_overview.ipynb)<br>